In [1]:
# -*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import jieba
import jieba.analyse
import codecs

# Seperate words for the text

In [2]:
from collections import Counter
import jieba
jieba.load_userdict('userdict.txt')

# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stoped.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

inputs = open('哈利波特与魔法石(人文版).txt', 'r',encoding="utf-8") #加载要处理的文件的路径
outputs = open('output.txt', 'w') #加载处理后的文件路径
for line in inputs:
    line_seg = seg_sentence(line)  # 这里的返回值是字符串
    outputs.write(line_seg)
outputs.close()
inputs.close()
# WordCount
with open('output.txt', 'r') as fr: #读入已经去除停用词的文件
    data = jieba.cut(fr.read())
data = dict(Counter(data))

with open('cipin.txt', 'w') as fw: #读入存储wordcount的文件路径
    for k, v in data.items():
        fw.write('%s,%d\n' % (k, v))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/89/qp8kb90s49z7xcdskq2c0qlm0000gn/T/jieba.cache
Loading model cost 0.666 seconds.
Prefix dict has been built succesfully.


In [3]:
# 拆分句子
# 版本为python3，如果为python2需要在字符串前面加上u
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('([，！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")


# Calculate the length of the sentenct

In [4]:
dict_sentence_len=dict()
inputs = open('哈利波特与魔法石(人文版).txt', 'r',encoding="utf-8") #加载要处理的文件的路径
for line in inputs: #这个循环是遍历整个文本的，inputs是整个文本，line是文本中的每一段
    #print(line)    
    l=cut_sent(line)   # l是给一段文本进行拆分句子后的结果
    for w in l:  #这个循环是遍历一段文字里的每个句子,l是一段里的所有句子存放的list，w是每个句子
        line_seg = seg_sentence(w)  #line_seg 是一个句子拆分词语后的结果
        words=list(line_seg.split(' ')) #将一个句子中的词语拆分后逐个存放在list中
        dict_sentence_len[line_seg]=len(words)+1  #本句的长度为这个list的长度（+1是因为句子从0开始）
        #print(words)
        #print(len(words))

# Transform the dict into dataframe and save the results of sentence-length into .xls file

In [5]:
df_sentence_len=pd.DataFrame(columns = ['句子','长度']) #为了方便存储，我把dict转成了dataframe格式，最后存放在excel表格里
df_sentence_len.head()
df_sentence_len['句子']=dict_sentence_len.keys()
df_sentence_len['长度']=dict_sentence_len.values()
df_sentence_len.to_excel('句子长度.xls')

# Calculate characters

## Read character file

In [6]:
text = open('哈利波特与密室(人文版).txt', 'r',encoding="gbk") #加载要处理的文件的路径
roles_df=pd.read_excel('character.xlsx',names=['character'],index=[0])
roles_df.head()

,character
0,邓布利多
1,亚瑟
2,韦斯莱
3,巴特缪斯
4,巴蒂


# Calculate the frequency of roles in certain length text

In [7]:
def calculate_roles(inputs):
    roles_dict=dict()
    for line in inputs:
        line_seg = seg_sentence(line)
        words_list=list(line_seg.split(' '))
        print(words_list)
        for role in roles_df['character']:
            #print(role)
            if role in words_list:
                if role not in roles_dict:
                    roles_dict[role]=1
                else:
                    roles_dict[role]+=1
                #print(role)
    return roles_dict
    

In [8]:
# Calculate roles of the whole text
calculate_roles(text)

['哈利波特', '密室', '人文', '社', '']
['']
['仅仅', '灾难', '序曲', '多比', '预言', '哈利', '遭受', '重重', '磨难', '经历', '种种', '危险', '难解之谜', '使', '熬', '费股', '新', '霍格沃茨', '学生', '接二连三', '变成', '石头', '使', '品德', '败坏', '学生', '德拉科', '马尔福', '精心策划', '杰作', '忠厚老实', '']
['']
['作者', '']
['J', 'K', '罗琳', '', '', '著', '']
['出版社', '']
['人民文学出版社', '']
['']
['']
['']
['目录', '']
['']
['主要', '人物', '表', '']
['1', '章', '', '', '最', '糟糕', '生日', '']
['2', '章', '', '', '多比', '警告', '']
['3', '章', '', '', '陋居', '']
['4', '章', '', '', '在丽痕', '书店', '']
['5', '章', '', '', '打人柳', '']
['6', '章', '', '', '吉', '德罗', '洛', '哈特', '']
['7', '章', '', '', '泥巴', '细语', '']
['8', '章', '', '', '忌辰', '晚会', '']
['9', '章', '', '', '墙上', '字', '']
['10', '章', '', '', '失控', '游走', '球', '']
['11', '章', '', '', '决斗', '俱乐部', '']
['12', '章', '', '', '复方', '汤剂', '']
['13', '章', '', '', '绝密', '日记', '']
['14', '章', '', '', '康奈利福吉', '']
['15', '章', '', '', '阿拉戈', '克', '']
['16', '章', '', '', '密室', '']
['17', '章', '', '', '斯莱特林', '继承人', '']
['18', '章', '', '', '多比', '报

['能换', '霍格沃茨', '一点', '音信', '来自', '女巫', '男巫', '都', '会', '豁出去', '乐意', '看', '一眼', '仇敌', '德拉科', '马尔福', '证明', '不是', '一场', '梦', '..', '霍格沃茨', '一年', '不', '都', '好玩', '有趣', '上', '学期末', '哈利', '伏地魔', '本人', '正面', '相遇', '伏地魔', '大', '从前', '依然', '狠毒', '可怕', '阴险', '狡猾', '决心', '恢复', '魔力', '哈利', '一次', '逃脱', '伏地魔', '魔爪', '很险', '现在', '已经', '过去', '好几个', '星期', '哈利', '还会', '半夜', '惊醒', '浑身', '冷汗', '想着', '伏地魔', '会', '记起', '青灰色', '脸', '圆', '睁', '疯狂', '眼睛', '..', '哈利', '突然', '坐直', '身子', '心不在焉', '注视', '树篱', '', '', '现在', '树篱', '正', '注视', '树叶', '丛中', '闪动', '两只', '大得', '出奇', '绿', '眼睛', '']
['']
['哈利', '跳', '起来', '草坪', '对面', '飘过来', '一声', '嘲笑', '']
['']
['知道', '今天', '日子', '达力', '摇摇摆摆', '走', '过来', '']
['']
['大', '眼睛', '忽闪', '几下', '消失', '']
['']
['哈利', '说', '眼睛', '还', '盯', '地方', '']
['']
['知道', '今天', '日子', '达力', '说', '一遍', '走到', '旁边', '']
['']
['很', '好', '哈利', '说', '终于', '学会', '数', '星期', '']
['']
['今天', '生日', '!', '达力', '讥讽', '地说', '居然', '没有', '收到', '贺卡', '鬼', '地方', '朋友', '都', '没有', '']
['']
['最好', '妈妈', '听到', '说', '学校

['活', '板门', '一响', '佩妮', '姨妈', '手', '洞口', '推进', '一碗', '罐头', '汤', '哈利', '早就', '饿得', '肚子疼', '赶紧', '跳下床', '捧起', '只', '碗', '汤是', '冰凉', '一口气', '喝下', '半碗', '走', '海德薇', '笼子', '旁', '碗底', '几根', '泡', '水', '蔬菜', '倒进', '空空', '食盘', '里', '竖起', '羽毛', '充满', '厌恶', '看', '一眼', '']
['']
['鸟嘴', '翘', '老高', '哈利', '板着脸', '说', '空碗', '放', '回活', '板门', '旁', '重新', '躺', '床上', '感觉', '喝汤', '前', '更', '饿', '假设', '四星期', '后', '还', '活着', '却', '没', '去', '霍格沃茨', '报', '那会', '会', '不会', '派人来', '调查', '没', '回去', '使', '德思礼', '一家', '放', '走', '屋里', '黑', '下来', '哈利', '精疲力竭', '饥肠辘辘', '脑子里', '翻来覆去', '地转', '没有', '答案', '问题', '不知不觉', '睡着', '睡得', '很', '不', '安稳', '']
['']
['梦见', '放在', '动物园', '里', '展览', '笼子', '上', '卡片', '写', '小巫师', '人们', '隔', '铁栅栏', '看', '躺', '稻草', '上', '饿得', '奄奄一息', '人群', '中', '看到', '多比', '面孔', '忙', '喊', '救', '多比', '道', '哈利波特', '安全', '先生', '!', '说完', '消失', '看到', '德思礼', '一家', '达力', '摇着', '铁笼', '栏杆', '嘲笑', '']
['']
['住手', '哈利', '含糊不清', '地说', '嘎啦', '嘎啦', '声音', '震动', '疼痛', '神经', '别吵', '..', '停下', '..', '想', '睡觉', '..', '睁开眼', '月

['']
['魔杖', '水池', '里', '碗碟', '随意', '一点', '碗碟', '清洗', '起来', '叮叮当当', '19', '', '', '声音', '像是', '一种', '背景音乐', '']
['']
['情况', '很', '不好', '妈妈', '!', '弗雷德', '说', '']
['']
['吃饭', '不要', '说话', '!', '韦斯莱', '夫人', '厉声', '说', '']
['']
['不', '饭', '吃', '妈妈', '!', '乔治', '说', '']
['']
['闭嘴', '!', '韦斯莱', '夫人', '说', '动手', '哈利', '切', '面包', '涂', '黄油', '时', '脸上', '表情', '已', '稍稍', '温和', '']
['']
['穿着', '长', '睡衣', '红头发', '小人儿', '跑', '进', '厨房', '尖叫', '一声', '跑', '出去', '金妮', '罗恩', '低声', '哈利', '说', '妹妹', '暑假', '都', '念叨', '', '', '可不', '想要', '签名', '哈利', '弗雷德', '笑', '道', '看到', '母亲', '眼神', '马上', '埋头', '吃饭', '不再', '说话', '几人', '闷声不响', '不一会儿', '四只', '盘子', '便', '一扫而空', '好累', '弗雷德', '放下', '刀叉', '说', '想', '去', '睡觉', '', '', '', '', '不行', '韦斯莱', '夫人', '无情地', '说', '晚上', '没睡', '自找', '现在', '去', '清除', '花园里', '精', '闹', '不可收拾', '']
['']
['妈妈', '', '', '']
['']
['两个', '瞪', '罗恩', '弗雷德', '说', '哈利', '说', '去', '睡觉', '亲爱', '没有', '开', '那辆', '破车', '']
['']
['哈利', '觉得', '一点儿', '不困', '忙', '说', '帮', '罗恩', '一块儿', '干', '还', '没见', '清除', '地精'

['没', '问题', '妈妈', '弗雷德', '说', '哈利', '先看', '做', '']
['']
['花盆里', '捏起', '撮', '亮晶晶', '粉末', '走', '火炉', '前', '粉末', '丢进', '火焰', '里', '']
['']
['呼', '一声', '炉火', '变得', '碧绿', '升得', '弗雷德', '还', '高', '径直', '走进', '火', '里', '喊', '一声', '对角巷', '!', '眨眼', '间', '不见', '']
['']
['必须', '几个', '字', '说', '清楚', '孩子', '韦斯莱', '夫人', '哈利', '说', '乔治', '把手', '仲进', '花盆', '出来', '时', '千万别', '走', '错', '炉门', '..', '']
['']
['走', '错', '哈利', '紧张', '地问', '火焰', '呼啸', '蹿', '乔治', '卷', '走', '']
['']
['知道', '魔火', '很', '多种', '必须', '选准', '口齿清楚', '', '', '']
['']
['不会', '有事', '莫丽', '紧张', '兮兮', '韦斯莱', '先生', '说', '取', '飞路粉', '亲爱', '走', '丢', '姨父', '姨妈', '交代', '不会', '着急', '哈利', '安慰', '说', '达力', '会', '觉得', '烟囱', '里', '失踪', '绝妙', '笑话', '不用', '担心', '..', '好', '..', '亚瑟', '后面', '走', '韦斯莱', '夫人', '说', '记住', '走进', '火', '里', '说', '去', '', '', '']
['']
['胳膊肘', '夹紧', '罗恩', '教他', '']
['']
['闭上眼睛', '韦斯莱', '夫人', '说', '煤烟', '', '', '']
['']
['不要', '乱动', '罗恩', '说', '可能', '炉门', '跌', '出来', '', '', '']
['']
['27', '', '', '不要', '慌里慌张', '不要', '出来', '得太

...

['想想', '说', '东西部', '收拾', '齐', '噢', '还', '忘', '一件', '事情', '']
['']
['抽出', '魔杖', '转向', '哈利', '罗恩', '']
['']
['由衷', '抱歉', '孩子', '不得不', '施', '遗忘', '魔咒', '不能', '秘密', '到处', '张扬', '不然的话', '书', '想', '卖出去', '..', '哈利', '及时', '拔出', '魔杖', '洛', '哈特', '刚', '魔杖', '举起', '哈利', '大', '吼', '一声', '武器', '!', '洛', '哈特', '被击', '倒退', '几步', '摔倒', '皮箱', '上', '魔杖', '高高地', '飞', '空中', '罗恩', '接', '住', '扔', '敞开', '窗户', '外面', '去', '']
['']
['不', '应该', '斯内普', '教授', '教', '咒语', '哈利', '气愤', '地说', '一脚', '洛', '哈特', '箱子', '踢', '一边', '洛啥', '特', '抬头', '看着', '邓', '模样', '显得', '更', '枯瘦', '憔悴', '哈利', '仍然', '魔杖', '指着', '']
['']
['想要', '做', '洛', '哈特', '虚弱', '地说', '可不', '知道', '密室', '不会', '算', '运气', '好', '哈利', '说', '魔杖', '指着', '洛', '哈特', '强迫', '站', '起身', '碰巧', '知道', '密室', '还', '知道', '密室', '里关', '走', '押', '洛', '哈特', '走出', '办公室', '最近', '一道', '楼梯', '下去', '走过', '墙上', '闪着', '文字', '昏暗', '走廊', '来到', '哭泣的桃金娘', '盥洗室', '门口', '派洛', '哈特', '走', '最', '前面', '哈利', '开心', '看见', '浑身发抖', '哭泣的桃金娘', '正', '坐在', '最', '里面', '抽水马桶', '水箱', '上', '噢', '看见', '哈利'

['意思', '说', '快点', '魔杖', '可能', '会', '需要', '']
['']
['里德尔', '笑容', '更', '明显', '']
['']
['不会', '需要', '说', '']
['']
['哈利', '吃惊', '地望', '']
['']
['说', '不会', '', '', '']
['']
['哈利', '波特', '很', '长时间', '里德尔', '说', '希望', '机会', '看到', '谈谈', '']
['']
['哈利', '渐渐', '失去', '耐心', '说道', '大概', '还', '没有', '明白', '意思', '现在', '密室', '里', '不妨', '以后', '再谈', '']
['']
['必须', '现在', '谈', '里德尔', '说', '脸上', '仍', '挂', '明显', '笑容', '哈利', '魔杖', '揣进', '口袋', '']
['']
['哈利', '目瞪口呆', '看着', '发生', '事情', '真是太', '古怪', '']
['']
['金妮', '会', '变成', '慢慢', '地问', '']
['']
['有趣', '问题', '里德尔', '愉快', '地说', '说来话长', '看', '金妮', '韦斯莱', '会', '变成', '真正', '原因', '看不见', '陌生人', '敞开', '心扉', '倾诉', '全部', '秘密', '']
['']
['说些', '哈利', '说', '']
['']
['日记', '里德尔', '说', '日记', '好几个', '月', '小金', '妮', '上面', '写', '心里话', '诉说', '令人', '心疼', '烦恼', '悲哀', '哥哥', '取笑', '不得不', '穿着', '旧', '长袍', '旧书', '上学', '认为', '', '', '里德尔', '眼睛', '狡猾地', '闪烁着', '', '', '认为', '大名鼎鼎', '善良', '伟大', '哈利波特', '永远', '不会', '喜欢', '..', '']
['']
['里德尔', '说话', '目光', '始终', '没有', '离开', '哈利', '脸', '眼睛

['哈利', '墙壁', '滑', '地上', '抓住', '那根', '正在', '身体', '里', '喷射', '毒液', '长牙', '胳膊', '里拔', '出来', '知道', '已经', '晚', '剧烈', '疼痛', '正', '缓慢', '持续', '伤口', '全身', '蔓延', '扔掉', '毒牙', '注视', '鲜血', '慢', '馒', '浸透', '长袍', '时', '视线', '已经', '模糊', '密室', '逐渐', '消融', '一团', '飞速', '旋转', '昏暗', '色彩', '中', '']
['']
['一道', '鲜红色', '光', '轻盈', '眼前', '掠过', '哈利', '听见', '身边', '传来', '爪子', '轻轻', '抓挠', '声', '']
['']
['福克斯', '哈利', '含混不清', '地说', '太棒了', '福克斯', '..', '感到', '大鸟', '美丽', '脑袋', '贴', '蛇', '怪', '毒牙', '刺中', '地方', '']
['']
['他昕', '见', '伴随', '回音', '脚步声', '黑压压', '影子', '站', '面前', '']
['']
['死', '哈利波特', '里德尔', '声音', '上面', '说', '死', '邓布利多', '鸟', '知道', '一点', '看见', '做', '', '', '波特', '哭', '']
['']
['哈利', '眨', '眨眼睛', '福克斯', '脑袋', '忽而', '清晰', '忽而', '模糊', '大滴', '大滴', '珍珠', '般的', '泪珠', '富有', '光泽', '羽毛', '滚落', '下来', '']
['']
['我要', '坐在', '亲眼', '看着', '死去', '哈利波特', '不要', '着急', '时间', '']
['']
['哈利', '感到', '昏昏欲睡', '周围', '似乎', '都', '旋转', '']
['']
['大名鼎鼎', '哈利波特', '完蛋', '里德尔', '声音', '很', '远', '地方', '传来', '孤零零', '密室', '里', '朋友', '抛弃', '不自量力'

['说', '应该', '斯莱特林', '哈利', '绝望', '盯', '邓布利多', '脸', '说道', '分院帽', '可能', '身上', '看到', '斯莱特林', '本领', '..', '']
['']
['放在', '格兰芬多', '邓布利多', '不紧', '不', '馒地', '说', '听', '说', '哈利', '碰巧', '具有', '萨拉查', '斯莱特林', '精心', '挑选', '学生', '时', '特别', '看重', '索质', '自已', '罕见', '天赋', '蛇佬腔', '..', '足智多谋', '..', '意志坚强', '..', '某种', '法律', '条规', '藐视', '说', '银白色', '胡须', '微微', '颤抖', '分院帽', '放在', '格兰芬多', '知道', '会', '好好', '想想', '']
['']
['放在', '格兰芬多', '利用', '一种', '心灰意冷', '口气', '说', '是因为', '提出', '不去', '斯莱特林', '..', '']
['']
['正是', '邓布利多', '说', '脸上', '露出', '笑容', '使', '汤姆', '里德尔', '大不一样', '哈利', '表现', '真正', '自我', '选择', '这比', '具有', '能力', '更', '重要', '哈利', '一动不动', '坐在', '椅子', '里', '完全', '呆住', '哈利', '还', '需要', '证据', '确信', '真的', '属于', '格兰芬多', '建议', '再', '仔细', '看看', '']
['']
['邓布利多', '探身', '麦格', '教授', '书桌上', '血迹斑斑', '银剑', '递给', '哈利', '哈利', '茫然', '翻过来', '红宝石', '火光', '映照', '下', '闪亮', '夺目', '看见', '靠近', '剑柄', '地方', '刻着', '名字', '']
['']
['戈德里', '克', '格兰芬多', '真正', '格兰芬多', '才能', '帽子', '里', '抽出来', '哈利', '邓布利多', '简单', '地说', '一时间', '两人', '没

{'多比': 86,
 '德拉科': 33,
 '马尔福': 302,
 '哈利': 1046,
 '海格': 108,
 '赫敏': 269,
 '纳威': 26,
 '伏地魔': 17,
 '汤姆': 17,
 '里德尔': 94,
 '韦斯莱': 157,
 '格兰杰': 23,
 '邓布利多': 119,
 '波特': 96,
 '卢修斯': 27,
 '亚瑟': 18,
 '胖夫人': 3,
 '斯普劳特教授': 20,
 '高尔': 21,
 '费尔奇': 44,
 '庞弗雷夫人': 27,
 '差点没头的尼克': 28,
 '皮皮鬼': 15,
 '血人巴罗': 1,
 '哭泣的桃金娘': 22,
 '宾斯教授': 15,
 '平斯夫人': 5,
 '马沃罗': 2}

# Read raw text

In [9]:
text = open('哈利波特与密室(人文版).txt', 'r',encoding="gbk") #加载要处理的文件的路径
chapter_df=pd.read_excel('chapters.xlsx',names=['chapter','','',''],index=[0])
chapter_df.head()

,chapter,,,
0,第2章,NaN,NaN,NaN
1,第3章,NaN,NaN,NaN
2,第4章,NaN,NaN,NaN
3,第5章,NaN,NaN,NaN
4,第6章,NaN,NaN,NaN


In [10]:
# Seperate

# seperate chapters and save all chapters into one list

In [11]:
i=0
one_chapter=list() #used to save all lines in single chapter
all_chapters=list() # used to save all chapters into one list
for line in text:        # read all lines
    if chapter_df['chapter'][i] in line: # identify the chapter mark
        #print(chapter_df['chapter'][i])
        all_chapters.append(one_chapter) # if true, we save the single chapter into the chapter list
        one_chapter=list() # then we initialize the chapter list
        i+=1
    one_chapter.append(line)      

# calculate character frequency in each chapter

In [12]:
for chaps in all_chapters:  
    print(calculate_roles(chaps))

['哈利波特', '密室', '人文', '社', '']
['']
['仅仅', '灾难', '序曲', '多比', '预言', '哈利', '遭受', '重重', '磨难', '经历', '种种', '危险', '难解之谜', '使', '熬', '费股', '新', '霍格沃茨', '学生', '接二连三', '变成', '石头', '使', '品德', '败坏', '学生', '德拉科', '马尔福', '精心策划', '杰作', '忠厚老实', '']
['']
['作者', '']
['J', 'K', '罗琳', '', '', '著', '']
['出版社', '']
['人民文学出版社', '']
['']
['']
['']
['目录', '']
['']
['主要', '人物', '表', '']
['1', '章', '', '', '最', '糟糕', '生日', '']
{'多比': 1, '德拉科': 1, '马尔福': 2, '哈利': 1}
['2', '章', '', '', '多比', '警告', '']
{'多比': 1}
['3', '章', '', '', '陋居', '']
{}
['4', '章', '', '', '在丽痕', '书店', '']
{}
['5', '章', '', '', '打人柳', '']
{}
['6', '章', '', '', '吉', '德罗', '洛', '哈特', '']
{}
['7', '章', '', '', '泥巴', '细语', '']
{}
['8', '章', '', '', '忌辰', '晚会', '']
{}
['9', '章', '', '', '墙上', '字', '']
{}
['10', '章', '', '', '失控', '游走', '球', '']
{}
['11', '章', '', '', '决斗', '俱乐部', '']
{}
['12', '章', '', '', '复方', '汤剂', '']
{}
['13', '章', '', '', '绝密', '日记', '']
{}
['14', '章', '', '', '康奈利福吉', '']
{}
['15', '章', '', '', '阿拉戈', '克', '']
{}
['16', '章

In [13]:
calculate_roles(chaps)

['17', '章', '', '', '斯莱特林', '继承人', '']


{}

In [14]:
df_sentence_len=pd.DataFrame(columns = ['角色','出现次数'])
df_sentence_len.head()
df_sentence_len['句子']=dict_sentence_len.keys()
df_sentence_len['长度']=dict_sentence_len.values()
df_sentence_len.to_excel('哈利波特与魔法石-句子长度.xls')

In [15]:
chapter_df=pd.read_excel('chapters.xlsx',names=['chapter','','',''],index=[0])
chapter_df.head()

,chapter,,,
0,第2章,NaN,NaN,NaN
1,第3章,NaN,NaN,NaN
2,第4章,NaN,NaN,NaN
3,第5章,NaN,NaN,NaN
4,第6章,NaN,NaN,NaN
